In [16]:
import pandas as pd
from tqdm import tqdm

In [17]:
folder = "data"
full_list = pd.DataFrame()
start_year = 1999
end_year = 2024

column_renames = {
    "notering": "positie",
    "nr.": "positie",
    "nr": "positie",
    "jaar": "jaartal",
    "year": "jaartal",
    "title": "titel",
    "artist": "artiest",
}

for year in tqdm(range(start_year, end_year+1)):
    df = pd.read_excel(f'{folder}/TOP-2000-{year}.xlsx')
    if len(df) > 2000:
        df = df.iloc[1:]
    df.columns = [str.lower(col) for col in df.columns]
    df.rename(columns=column_renames, inplace=True)
    df['lijst_jaar'] = year
    df['positie'] = df['positie'].astype(int)
    df['jaartal'] = df['jaartal'].astype(int)
    df['lijst_jaar'] = df['lijst_jaar'].astype(int)
    full_list = pd.concat([full_list, df], ignore_index=True)
full_list

100%|██████████| 26/26 [00:01<00:00, 24.71it/s]


,positie,titel,artiest,jaartal,lijst_jaar
0,1,Bohemian Rhapsody,Queen,1975,1999
1,2,Hotel California,Eagles,1977,1999
2,3,Child In Time,Deep Purple,1972,1999
3,4,Stairway To Heaven,Led Zeppelin,1971,1999
4,5,Paradise By The Dashboard Light,Meat Loaf,1978,1999
...,...,...,...,...,...
51995,1996,Once Upon A Time In The West,Dire Straits,1979,2024
51996,1997,Bobby Jean,Bruce Springsteen,1984,2024
51997,1998,De Langste Nacht,Goldband,2021,2024
51998,1999,It's The End Of The World As We Know It (Album...,R.E.M.,1987,2024


In [18]:
meta_data = pd.read_csv(f"{folder}/song_metadata.csv")
meta_data.head(1)

,duration,album,tags,artist,song
0,355000,A Night at the Opera,"['classic rock', 'rock', 'Queen']",Queen,Bohemian Rhapsody


In [19]:
merged = pd.merge(
    full_list,
    meta_data,
    how="left",
    left_on=["titel", "artiest"],
    right_on=["song", "artist"],
).drop(columns=["song", "artist"])
merged.head(1)

,positie,titel,artiest,jaartal,lijst_jaar,duration,album,tags
0,1,Bohemian Rhapsody,Queen,1975,1999,355000.0,A Night at the Opera,"['classic rock', 'rock', 'Queen']"


In [20]:
# Looking at the null values
merged.isnull().sum()

positie          0
titel            0
artiest          0
jaartal          0
lijst_jaar       0
duration       425
album         2749
tags           425
dtype: int64

In [21]:
merged["artiest"] = merged["artiest"].str.replace("The ", "")

In [22]:
merged["full_track"] = merged["titel"] + " - " + merged["artiest"]

In [23]:
def preprocess_full_track(track):
    track = track.lower().replace("(albumversie)", "")
    track = track.replace("(live)", "")
    track = track.replace("(singleversie)", "")
    return track

merged["track_id"] = merged["full_track"].apply(preprocess_full_track)
merged["track_id"].sample(3)

49939    hopeless wanderer - mumford & sons
37938     black magic woman - fleetwood mac
9613               photograph - ringo starr
Name: track_id, dtype: object

In [24]:
def build_track_id(track):
    return "".join(e for e in track if e.isalnum())

merged["track_id"] = merged["track_id"].apply(build_track_id)
merged["track_id"].sample(3)

33463      inbetweendayscure
42368              omarmbløf
21384    laislabonitamadonna
Name: track_id, dtype: object

In [25]:
# Dropping the full track column
merged.drop(columns=["full_track"], inplace=True)

In [26]:
folder = "data/final"
merged.to_parquet(f"{folder}/final.parquet", index=False)